# 01 · Exploración de Diccionarios, Datasets y Llaves

En esta sección se **escanea y perfila** los archivos colocados en `data/external/` para:
- Listar **datasets** (.xlsx, .xls, .csv, .parquet) por entidad: `obra/`, `empresa/`, `funcionario/`, `catalogos/`.
- Detectar **hojas** en Excel y previsualizar **cabeceras**.
- Inferir **esquemas** (tipos, nulos, conteo único) y **posibles llaves**.
- Exportar inventarios a CSV para documentación.

> Objetivo: Dejar definido **qué archivo(s)** usaremos como base para construir el *dataset maestro* de **Obra** y cuáles serán las **claves de unión** con Empresa y Funcionario.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import itertools

BASE = Path('..').resolve().parent / 'Deteccion_Corrupcion'
DATA_EXT = BASE / 'data' / 'external'
DATA_PROC = BASE / 'data' / 'processed'
DATA_EXT, DATA_PROC

(WindowsPath('D:/IA_Investigacion/Deteccion_Corrupcion/Deteccion_Corrupcion/data/external'),
 WindowsPath('D:/IA_Investigacion/Deteccion_Corrupcion/Deteccion_Corrupcion/data/processed'))

## 1) Escaneo de archivos por entidad
Se buscarán extensiones: `.xlsx`, `.xls`, `.csv`, `.parquet` en carpetas:
- `obra/`, `empresa/`, `funcionario/`, `catalogos/`

In [2]:
from typing import List, Dict

def scan_files(root: Path, sub: str) -> pd.DataFrame:
    folder = (root / sub)
    exts = ['*.xlsx','*.xls','*.csv','*.parquet']
    rows = []
    for pat in exts:
        for p in folder.rglob(pat):
            rows.append({
                'entidad': sub,
                'archivo': str(p.relative_to(root)),
                'nombre': p.name,
                'extension': p.suffix.lower(),
                'tamano_kb': round(p.stat().st_size/1024,2)
            })
    return pd.DataFrame(rows)

dfs = [scan_files(DATA_EXT, sub) for sub in ['obra','empresa','funcionario','catalogos']]
inv = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
inv.sort_values(['entidad','extension','archivo']).reset_index(drop=True)

,entidad,archivo,nombre,extension,tamano_kb
0,catalogos,catalogos\Diccionario vistas dashboards y data...,Diccionario vistas dashboards y datasets.xlsx,.xlsx,17.20
1,catalogos,catalogos\Diccionario_Datos_Sistemas_Fuente_V1...,Diccionario_Datos_Sistemas_Fuente_V1.xlsx,.xlsx,132.30
2,catalogos,catalogos\diccionario_campos.xlsx,diccionario_campos.xlsx,.xlsx,31.01
3,empresa,empresa\DS_DASH_Empresa_1A.csv,DS_DASH_Empresa_1A.csv,.csv,81.40
4,empresa,empresa\DS_DASH_Empresa_1B.csv,DS_DASH_Empresa_1B.csv,.csv,190.70
5,empresa,empresa\DS_DASH_Empresa_2A.csv,DS_DASH_Empresa_2A.csv,.csv,35.64
6,empresa,empresa\DS_DASH_Empresa_2B.csv,DS_DASH_Empresa_2B.csv,.csv,486.72
7,empresa,empresa\DS_DASH_Empresa_2C.csv,DS_DASH_Empresa_2C.csv,.csv,0.51
8,empresa,empresa\Datos generales - Empresa -2023.05.15....,Datos generales - Empresa -2023.05.15.xlsx,.xlsx,20.95
9,empresa,empresa\perfilamiento_empresa_riesgosa.xlsx,perfilamiento_empresa_riesgosa.xlsx,.xlsx,27.08


## 2) Guardar inventario de archivos
Se exporta un CSV con el listado completo para control documental.

In [3]:
out_inv = DATA_EXT / 'inventario_external_datasets.csv'
inv.to_csv(out_inv, index=False, encoding='utf-8')
out_inv, inv.shape

(WindowsPath('D:/IA_Investigacion/Deteccion_Corrupcion/Deteccion_Corrupcion/data/external/inventario_external_datasets.csv'),
 (26, 5))

## 3) Utilitarios de lectura segura
Lectura de Excel/CSV/Parquet y utilitarios para previsualizar esquemas y llaves candidatas.

In [4]:
def leer_tabla(path: Path, sheet_name=None, nrows=None) -> pd.DataFrame:
    if not path.exists():
        print(f'[WARN] No existe: {path}')
        return pd.DataFrame()
    if path.suffix.lower() == '.csv':
        return pd.read_csv(path, nrows=nrows)
    if path.suffix.lower() in ('.xlsx','.xls'):
        return pd.read_excel(path, sheet_name=sheet_name, nrows=nrows)
    if path.suffix.lower() == '.parquet':
        return pd.read_parquet(path)
    print(f'[WARN] Extensión no soportada: {path.suffix}')
    return pd.DataFrame()

OBRA_KEYS = ['id_obra','id_snip','codigo_unico','id_seace','id_proyecto']
RUC_KEYS  = ['ruc_empresa','ruc','ruc_proveedor']
DNI_KEYS  = ['dni_funcionario','dni','id_miembro','doc_identidad']

def detectar_llaves(df: pd.DataFrame) -> Dict[str, list]:
    keys = {'obra':[], 'ruc':[], 'dni':[]}
    cols = [c.lower() for c in df.columns]
    for k in OBRA_KEYS:
        if k in cols: keys['obra'].append(k)
    for k in RUC_KEYS:
        if k in cols: keys['ruc'].append(k)
    for k in DNI_KEYS:
        if k in cols: keys['dni'].append(k)
    return keys

def perfil_basico(df: pd.DataFrame, max_cols=20) -> pd.DataFrame:
    prof = pd.DataFrame({
        'dtype': df.dtypes.astype(str),
        'n_nulos': df.isna().sum(),
        'pct_nulos': df.isna().mean().round(4),
        'n_unicos': df.nunique(dropna=False)
    })
    return prof.sort_values(['pct_nulos','n_unicos'], ascending=[False, True]).head(max_cols)

## 4) Exploración por entidad
Para cada entidad, se mostrará un resumen por archivo y, en Excel, las hojas detectadas.

In [ ]:
def listar_hojas_excel(path: Path) -> list:
    try:
        xls = pd.ExcelFile(path)
        return xls.sheet_names
    except Exception as e:
        print('[WARN] No se pudo leer hojas:', path.name, e)
        return []

resumen = []
for _, row in inv.iterrows():
    p = DATA_EXT / row['archivo']
    sheets = []
    if row['extension'] in ('.xlsx','.xls'):
        sheets = listar_hojas_excel(p)
    resumen.append({
        'entidad': row['entidad'],
        'archivo': row['archivo'],
        'extension': row['extension'],
        'tamano_kb': row['tamano_kb'],
        'sheets': ', '.join(sheets[:10]) if sheets else ''
    })
df_resumen = pd.DataFrame(resumen).sort_values(['entidad','extension','archivo']).reset_index(drop=True)
df_resumen.head(30)

## 5) Guardar resumen y elegir muestras para *preview*
Se exporta `resumen_external_datasets.csv`. Abajo se puede configurar una lista corta de archivos a *previsualizar* (cabecera + perfil).

In [6]:
out_resumen = DATA_EXT / 'resumen_external_datasets.csv'
df_resumen.to_csv(out_resumen, index=False, encoding='utf-8')
out_resumen, df_resumen.shape

(WindowsPath('D:/IA_Investigacion/Deteccion_Corrupcion/Deteccion_Corrupcion/data/external/resumen_external_datasets.csv'),
 (26, 5))

### Archivos a previsualizar (ajusta esta lista)

In [9]:
preview_files = [
    'obra/perfilamiento_obra_riesgosa.xlsx',
    'empresa/perfilamiento_empresa_riesgosa.xlsx',
    'funcionario/perfilamiento_funcionario_riesgoso.xlsx',
]
preview_files

['obra/perfilamiento_obra_riesgosa.xlsx',
 'empresa/perfilamiento_empresa_riesgosa.xlsx',
 'funcionario/perfilamiento_funcionario_riesgoso.xlsx']

## 6) Preview: cabeceras, perfiles y llaves candidatas
Para los archivos listados en `preview_files`, se hace una lectura parcial y se muestran perfiles básicos y llaves probables.

In [8]:
perfiles_export = []
for rel in preview_files:
    p = DATA_EXT / rel
    if not p.exists():
        print('[WARN] No existe para preview:', rel)
        continue
    try:
        sheet = 0 if p.suffix.lower() in ('.xlsx','.xls') else None
        df = leer_tabla(p, sheet_name=sheet, nrows=2000)
        if df.empty:
            print('[WARN] Vacío o no legible:', rel)
            continue
        display(df.head(5))
        prof = perfil_basico(df, max_cols=30)
        display(prof)
        keys = detectar_llaves(df)
        print('Llaves candidatas ->', keys)
        tmp = prof.reset_index().rename(columns={'index':'columna'})
        tmp.insert(0, 'archivo', rel)
        perfiles_export.append(tmp)
    except Exception as e:
        print('[ERROR] Preview falló en', rel, e)

if perfiles_export:
    df_perf = pd.concat(perfiles_export, ignore_index=True)
    out_perf = DATA_EXT / 'perfiles_preview.csv'
    df_perf.to_csv(out_perf, index=False, encoding='utf-8')
    out_perf

[WARN] No existe para preview: obra/Perfilamiento Obra Riesgosa_CATEGORÍA X_2025.05.12.xlsx
[WARN] No existe para preview: empresa/Perfilamiento de Empresa riesgosa_ CATEGORÍA X _2025.05.12.xlsx
[WARN] No existe para preview: funcionario/Perfilamiento de Funcionario riesgoso_ CATEGORÍA X _2025.05.12.xlsx
